In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
from src.inference import get_feature_store

In [5]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-02-17 21:08:37,186 INFO: Initializing external client
2025-02-17 21:08:37,187 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 21:08:38,111 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213634
Fetching data from 2025-01-20 02:08:37.183905+00:00 to 2025-02-18 01:08:37.183905+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.53s) 


In [6]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-01-20 03:00:00+00:00,2,0
1,2025-01-20 04:00:00+00:00,2,0
2,2025-01-20 05:00:00+00:00,2,0
3,2025-01-20 06:00:00+00:00,2,0
4,2025-01-20 07:00:00+00:00,2,0
...,...,...,...
175577,2025-02-17 20:00:00+00:00,263,71
175578,2025-02-17 21:00:00+00:00,263,52
175579,2025-02-17 22:00:00+00:00,263,43
175580,2025-02-17 23:00:00+00:00,263,18


In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175582 entries, 0 to 176340
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175582 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175582 non-null  int32                  
 2   rides               175582 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [8]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [9]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175582 entries, 0 to 176340
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175582 non-null  datetime64[us]
 1   pickup_location_id  175582 non-null  int32         
 2   rides               175582 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [10]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [11]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,95,92,53,155,145,9,60,80,6,...,103,142,155,13,90,127,72,111,263,2025-02-15 17:00:00
1,161,154,87,114,94,234,167,83,100,53,...,226,129,243,45,111,229,406,5,132,2025-01-26 05:00:00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,147,2025-01-24 18:00:00
3,14,1,44,1,33,0,48,6,77,2,...,6,18,2,2,65,15,25,26,13,2025-01-30 02:00:00
4,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,155,2025-02-08 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,267,92,250,7,144,364,32,271,79,17,...,482,20,425,2,2,72,3,39,236,2025-01-24 06:00:00
249,1,0,0,1,0,2,1,0,0,0,...,0,0,0,0,0,1,0,0,174,2025-02-07 02:00:00
250,0,0,1,0,1,0,1,1,2,1,...,1,0,1,0,0,0,1,0,167,2025-01-29 02:00:00
251,66,158,1,98,0,105,12,69,47,2,...,49,75,158,77,72,5,24,125,246,2025-01-29 10:00:00


In [12]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-02-17 21:09:07,441 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-17 21:09:07,451 INFO: Initializing external client
2025-02-17 21:09:07,451 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 21:09:08,255 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213634
Fetching data from 2025-01-20 02:09:07.441440+00:00 to 2025-02-18 01:09:07.441440+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.13s) 


In [13]:
current_date

Timestamp('2025-02-18 02:09:07.441440+0000', tz='Etc/UTC')

In [14]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-17 03:00:00
1,0,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,3,2025-02-17 03:00:00
2,0,0,0,2,5,9,1,0,5,1,...,9,1,2,2,7,9,6,7,4,2025-02-17 03:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-17 03:00:00
4,2,1,1,2,2,4,9,6,1,1,...,3,0,2,1,3,1,0,1,7,2025-02-17 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,0,0,2,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,259,2025-02-17 03:00:00
249,1,1,0,2,2,3,0,3,1,0,...,0,3,2,0,0,1,0,1,260,2025-02-17 03:00:00
250,0,0,1,2,5,7,15,15,15,16,...,26,13,9,6,5,2,1,1,261,2025-02-17 03:00:00
251,1,4,11,70,134,140,153,98,91,101,...,29,20,14,14,9,5,5,0,262,2025-02-17 03:00:00


In [15]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-17 21:09:26,420 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-17 21:09:26,424 INFO: Initializing external client
2025-02-17 21:09:26,425 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 21:09:27,130 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213634

In [16]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

2025-02-17 21:09:31,069 WARNING: DeprecationWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.



In [17]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,1.0
...,...,...
248,259,0.0
249,260,1.0
250,261,1.0
251,262,2.0


In [18]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([ 79, 114, 249,  48, 246, 186, 230, 132, 125,  68])